This is a conversion of the [CUBA example](https://brian2.readthedocs.io/en/stable/examples/CUBA.html) from using Brian to pure Python + numpy. The code below is more or less equivalent to what Brian generates.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# In Brian, ms and mV have units, but here we simplify by defining them
# as constants
ms = mV = 0.001

# Neuron model parameters
taum = 20*ms         # membrane time constant
taue = 5*ms          # excitatory synapse time constant
taui = 10*ms         # inhibitory synapse time constant
Vt = -50*mV          # threshold
Vr = -60*mV          # reset
El = -49*mV          # reversal potential
refractory = 5*ms    # refractoriness

# Number of neurons
N = 4000             # number of neurons
Ne = int(N*0.8)      # excitatory neurons
Ni = N-Ne            # inhibitory neurons

# Synapse parameters
p = 0.02             # connection probability
we = (60*0.27/10)*mV # excitatory synaptic weight (voltage)
wi = (-20*4.5/10)*mV # inhibitory synaptic weight

# Simulation parameters
dt = 0.1*ms          # simulation time step
duration = 1000*ms    # simulation duration

# Convert times into timesteps
refractory_steps = round(refractory/dt)
duration_steps = round(duration/dt)

# Allocate memory for neuron state variables
v = np.zeros(N)
ge = np.zeros(N)
gi = np.zeros(N)
lastspike = np.zeros(N, dtype=int)

# Create synapses as a sparse matrix
J = []
for i in range(N):
  # For each source neuron i, pick a set of target neurons j with probability p
  j = (np.random.rand(N)<p).nonzero()[0]
  J.append(j)

# Now convert that into sparse matrix format, one long array of targets
# whose length is equal to the total number of synapses, and an offset array
# whose length is 1+N. Synapses from neuron i are stored in
#     targets[offsets[i]:offsets[i+1]]
# The following code converts from the list of arrays into this sparse format.

lengths = [len(j) for j in J]
targets = np.hstack(J)
offsets = np.hstack(([0], np.cumsum(lengths)))

# This is the code for updating the state variables from time t to t+dt
def state_update(timestep, v, ge, gi, lastspike):
  N = len(v)
  # Some constants that are the same for each neuron, so we calculate them
  # outside the loop. Note that Brian automatically finds these constants
  # from the set of input differential equations, and pulls them out of the
  # loop.
  alpha_m = np.exp(-dt/taum)
  alpha_e = np.exp(-dt/taue)
  alpha_i = np.exp(-dt/taui)
  tau_alpha_e = alpha_e*taue
  tau_alpha_i = alpha_i*taui
  cEl = El*(1-alpha_m)
  ce = ((tau_alpha_e * (-1/alpha_e + 1/alpha_m)) * alpha_m) / (taue-taum)
  ci = ((tau_alpha_i * (-1/alpha_i + 1/alpha_m)) * alpha_m) / (taui-taum)
  not_refractory = (timestep - lastspike[i] >= refractory_steps)
  _v = cEl + ce*ge + ci*gi + alpha_m*v
  # Don't update v if the cell is in a refractory state
  v[not_refractory] = _v[not_refractory]
  ge *= alpha_e
  gi *= alpha_i

# This is the code for propagating a set of spikes
def propagate_spikes(spiking, ge, gi):
  for i in spiking:
    # First Ne neurons are excitatory, others are inhibitory. We have a
    # different weight and different target variable depending on this.
    if i<Ne:
      g = ge
      w = we
    else:
      g = gi
      w = wi
    # Loop through all synapses for neuron i, and add weight to target
    # variable using sparse matrix structure
    g[targets[offsets[i]:offsets[i+1]]] += w

# Now run the simulation

# We use these to accumulate the recorded spikes so we can plot them at the end
spikes_i = []
spikes_t = []

# initialize values
v[:] = np.random.rand(N)*(Vt-Vr)+Vr
lastspike[:] = -(refractory_steps+1)
ge[:] = 0
gi[:] = 0

# Run through all timesteps
for timestep in range(duration_steps):
  # Step 1: update state variables (solve differential equations)
  state_update(timestep, v, ge, gi, lastspike)
  # Step 2: thresholding, which neurons have spiked?
  spiking = (v>Vt).nonzero()[0]
  # Step 3: propagate through synapses for those neurons
  propagate_spikes(spiking, ge, gi)
  # Step 4: reset membrane potentials for neurons that spiked
  v[spiking] = Vr
  lastspike[spiking] = timestep
  # Finally, record spikes
  spikes_i.append(spiking)
  spikes_t.append(np.full(len(spiking), timestep*dt))

# Convert recorded spikes into a single array for easy plotting
spikes_i = np.hstack(spikes_i)
spikes_t = np.hstack(spikes_t)

# And plot the spikes
plt.figure(figsize=(10, 6))
plt.plot(spikes_t/ms, spikes_i, ',k')
plt.xlabel('Time (ms)')
plt.ylabel('Neuron index')
plt.show();